In [1]:
import requests as req
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time
import random
import requests

In [2]:
info = {  # uninitialized dictionary to use for later
        'Name': [],
        'Price': [],
        'Image': [],
        'Link':[]
    }

In [3]:
url = 'https://attirstore.com/'

In [4]:
# Getting the navbar of the website
soup = bsp(req.get(url).content, 'html.parser')

# going into header with class "header header--middle-left header--mobile-center page-width header--has-menu header--has-social site-header-transparent" then going into navbar with class 'header__inline-menu'

navbar = soup.find('header', class_='header header--middle-left header--mobile-center page-width header--has-menu header--has-social site-header-transparent').find('nav', class_='header__inline-menu')

# going into ul class 'list-menu list-menu--inline' and getting all li then within the li tag there ar ul tag with class 'header__submenu list-menu list-menu--disclosure color-background-1 gradient caption-large motion-reduce global-settings-popup' and the getting all the href in the li tags

li = navbar.find_all('li')
links = []
for i in li:
    a = i.find('a')
    links.append(a['href'])

In [5]:
# printing all the links
for i in range(len(links)):
    links[i] = url + links[i]

In [6]:
# getting all the pages for each link if there exist one they are marked by "?page=2" where after = it can be any number from 2 to 10

AllLinks = []

for i in range(len(links)):
    AllLinks.append(links[i])
    for j in range(2,5):
        AllLinks.append(str(links[i])+f"?page={j}")


links = [link for link in AllLinks if "policies" not in link]

for link in links:
    print(link)

https://attirstore.com//collections/attir-man
https://attirstore.com//collections/attir-man?page=2
https://attirstore.com//collections/attir-man?page=3
https://attirstore.com//collections/attir-man?page=4
https://attirstore.com//collections/attir-man
https://attirstore.com//collections/attir-man?page=2
https://attirstore.com//collections/attir-man?page=3
https://attirstore.com//collections/attir-man?page=4
https://attirstore.com//collections/t-shirts
https://attirstore.com//collections/t-shirts?page=2
https://attirstore.com//collections/t-shirts?page=3
https://attirstore.com//collections/t-shirts?page=4
https://attirstore.com//collections/polo-shirts
https://attirstore.com//collections/polo-shirts?page=2
https://attirstore.com//collections/polo-shirts?page=3
https://attirstore.com//collections/polo-shirts?page=4
https://attirstore.com//collections/shirts
https://attirstore.com//collections/shirts?page=2
https://attirstore.com//collections/shirts?page=3
https://attirstore.com//collectio

In [7]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data

def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [8]:
# Getting all the products from the website and storing them in a dictionary using the links
# also changing the user agent to avoid getting blocked
proxy = get_valid_proxies()
print(len(proxy))

length = len(proxy)

6


In [9]:
TemoURL = 'https://attirstore.com'

In [10]:
for link in links:
    try:
        print(link)
        randomNumber = random.randint(0, length-1)
        randomNumber2 = random.randint(0, length-1)
        response = requests.get(link, headers=rotate_user_agent(proxy[randomNumber]['ip_address']))
        soup = bsp(response.text, 'html.parser')
        
        #check if there is a h2 with text 'No products found' if there is then skip the page
        h2 = soup.find('h2')
        print(h2)
        if h2.text is "No products found":
            continue
        
        # Each page has a ul with class 'grid__item scroll-trigger animate--slide-in' and within that ul there are li with class 'grid__item scroll-trigger animate--slide-in'
        ul = soup.find('ul', id='product-grid')
        li = ul.find_all('li', class_='grid__item scroll-trigger animate--slide-in')
        
        for i in li:
            try:
                headers = rotate_user_agent(proxy[randomNumber2]['ip_address'])  # Rotate user agent for each product
                # Each li will have a span with class 'price-item price-item--regular' within that will be price of the product
                span = i.find('span', class_='price-item price-item--regular')
                info['Price'].append(span.text.strip())
            
                # each li will have a div of class 'media media--transparent media--hover-effect' which will have a img tag with the image of the product
                div = i.find('div', class_='media media--transparent media--hover-effect')
                img = div.find('img')
                info['Image'].append(TemoURL+img['src'].strip())
            
                #each li will have a 'h3' with a href in it which will be the link to the product and within that h3 there will be a 'a' tag with the name of the product
                h3 = i.find('h3')
                a = h3.find('a')
                info['Link'].append(TemoURL+a['href'].strip())
                info['Name'].append(a.text.strip())
                
                time.sleep(5)  # Sleep after each product
            except Exception as e:
                print(f"An error occurred while processing a product: {e}")
    except Exception as e:
        print(f"An error occurred while processing the link {link}: {e}")

https://attirstore.com//collections/attir-man


<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Hp\AppData\Local\Temp\ipykernel_19776\1793142928.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if h2.text is "No products found":


<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man?page=2
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man?page=3
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man?page=4
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man?page=2
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man?page=3
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/attir-man?page=4
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/t-shirts
<h2 class="cart__empty-text">Your cart is empty</h2>
https://attirstore.com//collections/t-shirts?page=2
<h2 class="cart__empty-text">Your cart is empty</h2>
An error 

In [11]:
for i in info:
    print(info[i])

['Waffle Textured Shorts in Mustard', 'Waffle-Knit Classic Polo in Ice Blue', 'Waffle Textured Shorts in Grey Marl', 'Waffle-Knit Classic Polo in Cinnamon Brown', 'Waffle-Knit Classic Polo in Flamingo', 'Waffle-Knit Classic Polo in Khaki', 'Cotton Straight-Fit Trouser in Black', 'Waffle Textured Shirt in Beige', 'Yarn-Dyed Waffle Textured Shirt', 'Basic Cotton Shirt in Black', 'Stand-up Collar Shirt in Chocolate', 'Stand-up Collar Shirt in Olive', 'Stand-up Collar Shirt in Black', 'Waffle Textured Shorts in Cinnamon Brown', 'Stand-up Collar Shirt in White', 'Terry Sweatshorts in Cream', 'Terry Sweatshorts in Peach', 'Oversized T-shirt in Cream', 'Oversized T-shirt in Peach', 'Pull On Pants in Dark Navy', 'Pull On Pants in Cloud Grey', 'Pull On Pants in Olive', 'Creased-Effect Shirt in Black', 'Irish Cotton Textured Shirt in Black', 'Creased-Effect Shirt in Blue', 'Waffle-Knit Revere Polo Shirt in Rouge Pink', 'Waffle-Knit Revere Polo Shirt in Crimson', 'Polo Shirt with Revere Collar in

In [12]:
# converting the dictionary into a pandas dataframe and saving it as a csv file
df = pd.DataFrame(info)
df.to_csv('Attire.csv')